<a href="https://colab.research.google.com/github/siddartha1192/WeeklyOptionSellingCode/blob/master/fyers_option_chain_and_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install fyers-apiv3
# !pip install pendulum
# !pip install mibian

credentials

In [ ]:
client_id = '65L1HFF7DC-200'
secret_key = 'DKRVMFUG3U'
redirect_uri ='https://fessorpro.com/'

In [ ]:
access_token='eyJhbGc9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb1loZXplQ0I2bnFQOUdDODlTSlpKaDNHZEkwNWhkQ0xqYk1MRTR2RkdMY2hqNlZ6WG5jYW9VUGdPTHpYRWFYV1Q2bDBrTW04WjBHNHZfcWdOQWJBZWNEc2xXaDNoNHFvSjZxLWFaYzl1b2FjZU0zdz0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiYWI4MjcyNTE3ZGY0NDdmMGJlMDU3OTg1NjgyYzExMGE1MmNiZjJiMjM1YzhhY2E0ZjczYWM5ZCIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWFM0NTQ3NCIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzUxMzI5ODAwLCJpYXQiOjE3NTEyNTkwNTksImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc1MTI1OTA1OSwic3ViIjoiYWNjZXNzX3Rva2VuIn0.8Xe7b1RDhXLB9MHKxp3tUfDLvsEA6MjGkhtJ25Ayh3Q'
access_token

In [ ]:
from fyers_apiv3 import fyersModel
import pandas as pd
import pendulum as dt
import math
from scipy.stats import norm
import mibian
time_zone="Asia/Kolkata"

spot price

In [ ]:

# Initialize FyersModel instances for synchronous and asynchronous operations
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=None)
n='NIFTYBANK'
strike_diff=100
op_type='CE'
underlying=f"NSE:{n}-INDEX"

spot_price = fyers.quotes(data={"symbols":underlying}).get('d')[0].get('v').get('lp')
print('spot price is',spot_price)



expiry

In [ ]:
# Define the data for the option chain request
data = {
    "symbol": underlying,
    "strikecount": 5,
    "timestamp": ""
}

response = fyers.optionchain(data=data)['data']
expiry_e = response['expiryData'][0]['expiry']
expiry=dt.from_timestamp(int(expiry_e),tz=time_zone)
print("current expiry is",expiry)

option chain

In [ ]:
# Get the option chain data
response = fyers.optionchain(data=data)['data']
option_chain = pd.DataFrame(response['optionsChain'])
# option_chain

option name and strike

In [ ]:

# Function to get the OTM option based on spot price and side (CE/PE)



def get_otm_option(spot_price, side, points=0):
    if side == 'CE':
        otm_strike = (round(spot_price / strike_diff) * strike_diff) + points
    else:
        otm_strike = (round(spot_price / strike_diff) * strike_diff) - points
    otm_option = option_chain[(option_chain['strike_price'] == otm_strike) & (option_chain['option_type'] == side)]['symbol'].squeeze()
    return otm_option, otm_strike
strike_name,strike=get_otm_option(spot_price, op_type, points=0)
print('strike_name:',strike_name)
print('strike_price:',strike)

intrest rate

In [ ]:
ir=0.10
option_type =strike_name[-2:]
print(option_type)


time till expiry

In [ ]:
time_to_expiry = (expiry  - dt.now(time_zone)).total_seconds() / dt.duration(days=1).total_seconds()
print('time to expriy',time_to_expiry)

option price

In [ ]:
option_price = fyers.quotes(data={"symbols":strike_name}).get('d')[0].get('v').get('lp')
option_price

black scholes

In [ ]:
import math
from scipy.stats import norm

def black_scholes_price(S, K, T, r, sigma, option_type='CE'):
    """Calculate Black-Scholes price for call or put option.

    Optimizations:
    - Precompute common terms
    - Use more efficient math operations
    - Vectorized operations (though not applicable in this scalar case)
    """
    sqrt_T = math.sqrt(T)
    log_SK = math.log(S / K)
    r_sigma_sq = r + 0.5 * sigma * sigma

    d1 = (log_SK + r_sigma_sq * T) / (sigma * sqrt_T)
    d2 = d1 - sigma * sqrt_T

    if option_type == 'CE':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'PE':
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'CE' or 'PE'")

def implied_volatility(S, K, T, r, market_price, option_type='CE', tol=1e-6, max_iterations=100):
    """
    Calculate implied volatility using Newton-Raphson with optimizations:
    """
    # Handle edge cases first
    if market_price <= 0:
        return 0.0
    intrinsic_value = max(S - K, 0) if option_type == 'CE' else max(K - S, 0)
    if market_price <= intrinsic_value:
        return 0.0

    # Better initial guess based on Brenner-Subrahmanyam approximation for ATM options
    if S == K:
        sigma = market_price / (S * 0.3989 * math.sqrt(T))
    else:
        sigma = math.sqrt(2 * math.pi / T) * market_price / S

    sigma = max(sigma, 1e-4)  # Ensure we don't start with zero

    sqrt_T = math.sqrt(T)
    log_SK = math.log(S / K)
    discount = math.exp(-r * T)

    for i in range(max_iterations):
        sigma_sq = sigma * sigma
        r_sigma_sq = r + 0.5 * sigma_sq

        d1 = (log_SK + r_sigma_sq * T) / (sigma * sqrt_T)
        d2 = d1 - sigma * sqrt_T

        if option_type == 'CE':
            price = S * norm.cdf(d1) - K * discount * norm.cdf(d2)
        else:
            price = K * discount * norm.cdf(-d2) - S * norm.cdf(-d1)

        price_diff = price - market_price
        if abs(price_diff) < tol:
            return sigma

        vega = S * norm.pdf(d1) * sqrt_T
        if vega < 1e-12:  # Avoid division by very small numbers
            break

        sigma -= price_diff / vega
        sigma = max(sigma, 1e-4)  # Prevent negative volatility

    # Fall back to bisection if Newton-Raphson fails
    return _bisection_iv(S, K, T, r, market_price, option_type, tol, max_iterations)

def _bisection_iv(S, K, T, r, market_price, option_type, tol, max_iterations):
    """Fallback bisection method when Newton-Raphson fails"""
    low = 0.0
    high = 5.0  # 500% volatility as upper bound

    for i in range(max_iterations):
        mid = (low + high) * 0.5
        price = black_scholes_price(S, K, T, r, mid, option_type)

        if abs(price - market_price) < tol:
            return mid

        if price < market_price:
            low = mid
        else:
            high = mid

    return (low + high) * 0.5  # Return best estimate if not converged

iv = implied_volatility(spot_price, strike, time_to_expiry/365, ir, option_price, option_type=option_type)
print(f" IV: {iv:.4f}")

In [ ]:

mbo = mibian.BS([spot_price, strike, ir, time_to_expiry], volatility= iv*100)
delta = mbo.callDelta if option_type =="CE" else mbo.putDelta
theta =mbo.callTheta if option_type=="CE" else mbo.putTheta
vega =mbo.vega
gamma =mbo.gamma
print(f"spot_name: {underlying}, spot_price {spot_price}\nstrike_name {strike_name} strike price {option_price} expiry {expiry}")
print(f"iv :{iv} , delta :{delta} , theta: {theta}")